### Introduction: Business Problem
As an individual who is fascinated with very monumental things, It was not absurd that Joy Zel decided to move to Washington State in the United States of America. Known for being the place that houses the Space Needle, National Park and many others, it was a side attraction, she couldn’t turn down
As a center for Agriculture, Forestry and Fishery, Washington State contributes to the economy of the United States in a very important manner, and water being it’s most powerful resource. Washington is categorized by fertile soil and beautiful vegetation, and it is a contributing factor to the state’s importance
Washington State, also know as the State of Washington is a state in the Pacific Northwest Region of the United States. It is no surprise that it was named after the famous George Washington and has a population of 7.6 million. It is the 18th Largest state and the 13th most populous state.
The state’s coastal location and excellent harbours, give it a leading role in trade with Alaska, Canada, and countries of the Pacific Rim. Washington cities have sister cities in several countries, and their professional and trade associations commonly include Canadian members.
After the final decision on her relocated, there was one thing that had not yet been decided concerning Joy’s move which was: What City Would She Live In. There was quite a number of cities to choose from, but Joy wanted to be in the most optimal city and that is the problem we hope to solve.
The only condition she stipulated was that she wanted a city that was surrounded by popular tourist attractions such as; garden, landmarks, art museums and cafes. 
Using this information, we proceed to solving the problem.


### Data
In order to find the most suitable city for Joy, a list of cities with their corresponding zipcodes was downloaded from in csv format.
Venue queries will then be made by neighborhood name/cities using FourSquare Api. The resulting data regarding venue category will be used to find similarities in the corresponding cities. With a proper list of top venue categories for each state, a cluster can be made grouping them together for easier understanding. 
These are the data needed for the success of this project:
•	Cities and Zipcodes in Washington State
•	Coordinates of the cities
•	Top Venues for each city
•	Categories of Venue


### Data Gathering

## Import the important libraries

In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from geopy.geocoders import ArcGIS
import folium
import matplotlib.cm as cm
import matplotlib. colors as colors
import json

from IPython.display import JSON
from sklearn.cluster import KMeans

from pprint import pprint

## Read The Data File Downloaded

In [2]:
df=pd.read_csv("washingtonzipcode.csv")
df= df[["zip","City","County Name"]]

w_df= df.rename(columns={"City":"Neighborhood","zip":"Zip_Code"})

w_df.dropna(axis=0, inplace=True)

w_df


,Zip_Code,Neighborhood,County Name
0,98520,ABERDEEN,GRAYS HARBOR
1,98220,ACME,WHATCOM
2,99101,ADDY,STEVENS
3,98522,ADNA,LEWIS
4,99001,AIRWAY HEIGHTS,SPOKANE
...,...,...,...
688,98907,YAKIMA,YAKIMA
689,98908,YAKIMA,YAKIMA
690,98909,YAKIMA,YAKIMA
691,98597,YELM,THURSTON


## Install uszipcode which is an easy to use zip file in python

In [3]:
!pip install uszipcode
from uszipcode import SearchEngine


## Use the installed uszipcode to find the corresponding zip codes for Washington DC

In [4]:
search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in w_df.iterrows():
    zipcode = search.by_zipcode(row["Zip_Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

w_df["Latitude"] = latitude
w_df["Longitude"] = longitude
w_df.dropna(axis=0, inplace = True)
w_df

,Zip_Code,Neighborhood,County Name,Latitude,Longitude
0,98520,ABERDEEN,GRAYS HARBOR,47.100,-123.800
1,98220,ACME,WHATCOM,48.690,-122.160
2,99101,ADDY,STEVENS,48.400,-117.900
4,99001,AIRWAY HEIGHTS,SPOKANE,47.640,-117.580
5,99102,ALBION,WHITMAN,46.791,-117.255
...,...,...,...,...,...
686,98902,YAKIMA,YAKIMA,46.600,-120.530
687,98903,YAKIMA,YAKIMA,46.500,-120.800
689,98908,YAKIMA,YAKIMA,46.600,-120.800
691,98597,YELM,THURSTON,46.900,-122.400


In [6]:
w_df.loc[0, 'Neighborhood']

neighborhood_latitude = w_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = w_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = w_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ABERDEEN are 47.1, -123.8.


In [9]:
from geopy.geocoders import Nominatim

address = 'Washington DC'

geolocator = Nominatim(user_agent = "washinton_dc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Washington DC are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Washington DC are 38.8949924, -77.0365581.


## Using Folium, Generate A Map Using Latitudes and Longitude for Each Neighborhood and Insert Markers

In [103]:
map_w = folium.Map(location = [latitude, longitude], zoom_start=3)

# add markers to map
for lat, lng, neighborhood in zip(w_df['Latitude'], w_df['Longitude'], w_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'yellow',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_w)  
    
map_w

## Define FourSquare Api Credentials

In [112]:
CLIENT_ID = '***************' # your Foursquare ID
CLIENT_SECRET = '**********' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ***************
CLIENT_SECRET:**********


### Call The Foursquare Places Api To Find Venues In Washington DC

In [11]:
# Construct URL
radius = 500
LIMIT = 100
# Constructed API calls
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
# Request JSON
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f9bd4737b3a283282b2684b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Northwest Washington',
  'headerFullLocation': 'Northwest Washington, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 38.8994924045, 'lng': -77.03078704275774},
   'sw': {'lat': 38.8904923955, 'lng': -77.04232915724225}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb9e67c1261d13aeef4e998',
       'name': 'South Lawn',
       'location': {'address': '1600 Pennsylvania Ave NW',
        'lat': 38.89672777385296,
        'lng': -77.0366048812866,
        'label

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Import JSON. Read Venue Category and Store In "nearby_venues" variable

In [ ]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get Number Of Venues Returned

In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


In [16]:
washington_venues = getNearbyVenues(names=w_df['Neighborhood'],
                                   latitudes=w_df['Latitude'],
                                   longitudes=w_df['Longitude'])

ABERDEEN
ACME
ADDY
AIRWAY HEIGHTS
ALBION
ALLYN
ALMIRA
AMANDA PARK
AMBOY
ANACORTES
ANATONE
ANDERSON ISLAND
APPLETON
ARDENVOIR
ARIEL
ARLINGTON
ASHFORD
ASOTIN
AUBURN
AUBURN
AUBURN
BAINBRIDGE ISLAND
BATTLE GROUND
BAY CENTER
BEAVER
BELFAIR
BELLEVUE
BELLEVUE
BELLEVUE
BELLEVUE
BELLEVUE
BELLINGHAM
BELLINGHAM
BELLINGHAM
BENTON CITY
BEVERLY
BICKLETON
BINGEN
BLACK DIAMOND
BLAINE
BLAKELY ISLAND
BONNEY LAKE
BOTHELL
BOTHELL
BOTHELL
BOW
BREMERTON
BREMERTON
BREMERTON
BREMERTON
BREMERTON
BREWSTER
BRIDGEPORT
BRINNON
BRUSH PRAIRIE
BUCKLEY
BUCODA
BUENA
BURBANK
BURLINGTON
CAMANO ISLAND
CAMAS
CAMP MURRAY
CARBONADO
CARLTON
CARNATION
CARSON
CASHMERE
CASTLE ROCK
CATHLAMET
CENTERVILLE
CENTRALIA
CHATTAROY
CHEHALIS
CHELAN
CHELAN FALLS
CHENEY
CHEWELAH
CHIMACUM
CHINOOK
CINEBAR
CLALLAM BAY
CLARKSTON
CLAYTON
CLE ELUM
CLEARLAKE
CLINTON
COLBERT
COLFAX
COLLEGE PLACE
COLTON
COLVILLE
CONCONULLY
CONCRETE
CONNELL
CONWAY
COPALIS BEACH
COPALIS CROSSING
COSMOPOLIS
COUGAR
COULEE CITY
COULEE DAM
COUPEVILLE
COWICHE
CRESTON
CURLEW

In [17]:
print(washington_venues.shape)
washington_venues.head()

(26956, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABERDEEN,47.1,-123.8,South Lawn,38.896728,-77.036605,Garden
1,ABERDEEN,47.1,-123.8,The White House,38.897804,-77.036562,Government Building
2,ABERDEEN,47.1,-123.8,The Ellipse — President's Park South,38.894229,-77.036605,Park
3,ABERDEEN,47.1,-123.8,The West Wing,38.897488,-77.037560,Government Building
4,ABERDEEN,47.1,-123.8,W Hotel - Washington D.C.,38.897037,-77.033322,Hotel


## Group Washington Venues By Their Respective Neighbourhood

In [18]:
washington_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ABERDEEN,46,46,46,46,46,46
ACME,46,46,46,46,46,46
ADDY,46,46,46,46,46,46
AIRWAY HEIGHTS,46,46,46,46,46,46
ALBION,46,46,46,46,46,46
...,...,...,...,...,...,...
WOODLAND,46,46,46,46,46,46
YACOLT,46,46,46,46,46,46
YAKIMA,184,184,184,184,184,184


### Analysis

## Get The Number of Unique Categories and One-Hot Encode "Neighborhood" Column and Make It The First Column

In [19]:
print('There are {} uniques categories.'.format(len(washington_venues['Venue Category'].unique())))

There are 31 uniques categories.


In [83]:
# one hot encoding
wash_onehot = pd.get_dummies(washington_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
wash_onehot['Neighborhood'] = washington_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [wash_onehot.columns[-1]] + list(wash_onehot.columns[:-1])
wash_onehot = wash_onehot[fixed_columns]

wash_onehot.head()

,Neighborhood,American Restaurant,Art Museum,Bar,Café,Cocktail Bar,Coffee Shop,Concert Hall,Food Truck,French Restaurant,...,Nightlife Spot,Outdoor Sculpture,Park,Sandwich Place,Smoke Shop,Snack Place,Spa,Tour Provider,Tourist Information Center,Whisky Bar
0,ABERDEEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABERDEEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABERDEEN,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,ABERDEEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ABERDEEN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Group by Neighbourhood and Respective Venues

In [84]:
wash_grouped = wash_onehot.groupby('Neighborhood').mean().reset_index()
wash_grouped.head()

,Neighborhood,American Restaurant,Art Museum,Bar,Café,Cocktail Bar,Coffee Shop,Concert Hall,Food Truck,French Restaurant,...,Nightlife Spot,Outdoor Sculpture,Park,Sandwich Place,Smoke Shop,Snack Place,Spa,Tour Provider,Tourist Information Center,Whisky Bar
0,ABERDEEN,0.065217,0.043478,0.043478,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739
1,ACME,0.065217,0.043478,0.043478,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739
2,ADDY,0.065217,0.043478,0.043478,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739
3,AIRWAY HEIGHTS,0.065217,0.043478,0.043478,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739
4,ALBION,0.065217,0.043478,0.043478,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,...,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739


## Return Top 10 Venues In The Washington Area

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # append 'st', 'nd', 'rd' to the top 3 venues
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = wash_grouped['Neighborhood']

for ind in np.arange(wash_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wash_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABERDEEN,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
1,ACME,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
2,ADDY,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
3,AIRWAY HEIGHTS,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
4,ALBION,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place


## Import KMeans And Derive Clusters

In [88]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 1
wash_grouped_clustering = wash_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(wash_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [89]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

wash_merged = w_df
wash_merged = wash_merged.merge(neighborhoods_venues_sorted, on = 'Neighborhood')

wash_merged.head()

,Zip_Code,Neighborhood,County Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,98520,ABERDEEN,GRAYS HARBOR,47.100,-123.800,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
1,98220,ACME,WHATCOM,48.690,-122.160,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
2,99101,ADDY,STEVENS,48.400,-117.900,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
3,99001,AIRWAY HEIGHTS,SPOKANE,47.640,-117.580,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
4,99102,ALBION,WHITMAN,46.791,-117.255,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place


## Create Map With Corresponding Cluster(In this case: 1)

In [104]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(wash_merged['Latitude'], wash_merged['Longitude'], wash_merged['Neighborhood'], wash_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

### Result and Discussion

## Reviewing Cluster

It shows that after sorting venues by most common, the neighborhoods were grouped into just one cluster.Although this was not a possiblity we saw coming, it goes to show that this state that Joy has planned to relocate has a number of major requirements in a least a great percentage of the entire state. After reviewing the cluster, the cities analysed for this data presented that it has the major venue category requirement by Joy which was; cafe, art museum, garden. This shows that all the neighborhoods have the same concerntration of venue category.

### Cluster 1


In [107]:
wash_merged.loc[wash_merged['Cluster Labels'] == 0, wash_merged.columns[[1] + list(range(5, wash_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABERDEEN,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
1,ACME,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
2,ADDY,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
3,AIRWAY HEIGHTS,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
4,ALBION,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...,...
581,YAKIMA,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
582,YAKIMA,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
583,YAKIMA,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place
584,YELM,0,Garden,American Restaurant,Government Building,Hotel,Monument / Landmark,Art Museum,Bar,Café,Hotel Bar,Sandwich Place


## Cluster 2:This Cluster Has No Neighborhood

In [108]:
wash_merged.loc[wash_merged['Cluster Labels'] == 1, wash_merged.columns[[1] + list(range(5, wash_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Cluster 3:This Cluster Has No Neighborhood

In [110]:
wash_merged.loc[wash_merged['Cluster Labels'] == 2, wash_merged.columns[[1] + list(range(5, wash_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


### Conclusion

Joy can pick any of the cities cited in this study and she will be quite satisifed.